In [1]:
import pandas as pd

# Load each bank's stock price data
bank1 = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/Emirates NBD PJSC Stock Price History.csv')
bank2 = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/Dubai Islamic Bank Stock Price History.csv')
bank3 = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/First Abu Dhabi Bank Stock Price History (1).csv')
bank4 = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/Abu Dhabi Commercial Bank PJSC Stock Price History.csv')


# Add a 'Bank Name' column to differentiate datasets
bank1['Bank Name'] = 'ENBD'
bank2['Bank Name'] = 'DIB'
bank3['Bank Name'] = 'FAB'
bank4['Bank Name'] = 'ADCB'


# Combine all datasets into a single DataFrame
master_stock_data = pd.concat([bank1, bank2, bank3, bank4], ignore_index=True)

# Ensure Date is in proper format
master_stock_data['Date'] = pd.to_datetime(master_stock_data['Date'], errors='coerce')

# Sort the data by Date for consistency
master_stock_data = master_stock_data.sort_values(by=['Date', 'Bank Name'])

# Display the first few rows of the master dataset
print(master_stock_data.head())

# Save the combined dataset to a CSV file
master_stock_data.to_csv('master_stock_prices.csv', index=False)


           Date  Price   Open    High    Low     Vol. Change % Bank Name
9880 2014-12-17  6.000  5.550   6.250  5.050   10.83M    9.29%      ADCB
4913 2014-12-17  4.186  4.145   4.283  3.942   26.45M    2.00%       DIB
2412 2014-12-17  6.180  5.891   6.604  5.872    3.13M    4.91%      ENBD
7410 2014-12-17  9.860  9.090  10.000  9.090  662.52K    7.88%       FAB
9879 2014-12-18  6.900  6.400   6.900  6.140   12.71M   15.00%      ADCB


In [2]:

dfm_data = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/DFM General Historical Data.csv')
brent_data = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/Brent Oil Futures Historical Data(usd).csv')
usd_aed_data = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/USD_AED Historical Data.csv')

# Convert 'Vol.' in master_stock_data to numeric
def convert_volume(vol):
    if isinstance(vol, str):
        vol = vol.replace('K', 'e3').replace('M', 'e6').replace(',', '')
        try:
            return float(eval(vol))
        except:
            return None
    return vol

# Handle missing values in 'Vol.' column by converting units and filling with mean
master_stock_data['Vol.'] = master_stock_data['Vol.'].apply(convert_volume)
master_stock_data['Vol.'] = master_stock_data['Vol.'].fillna(master_stock_data['Vol.'].mean())

# Convert 'Change %' to numeric
master_stock_data['Change %'] = master_stock_data['Change %'].str.replace('%', '').astype(float)

# Add lagged features
master_stock_data['Price Lag 1'] = master_stock_data.groupby('Bank Name')['Price'].shift(1)
master_stock_data['Change % Lag 1'] = master_stock_data.groupby('Bank Name')['Change %'].shift(1)

# Add moving averages
master_stock_data['Price MA 5'] = master_stock_data.groupby('Bank Name')['Price'].rolling(window=5, min_periods=1).mean().reset_index(0, drop=True)
master_stock_data['Price MA 10'] = master_stock_data.groupby('Bank Name')['Price'].rolling(window=10, min_periods=1).mean().reset_index(0, drop=True)

# Create interaction term: Price per unit volume
master_stock_data['Price per Vol'] = master_stock_data['Price'] / master_stock_data['Vol.']

# Convert dates in external datasets to datetime
dfm_data['Date'] = pd.to_datetime(dfm_data['Date'], errors='coerce')
brent_data['Date'] = pd.to_datetime(brent_data['Date'], errors='coerce')
usd_aed_data['Date'] = pd.to_datetime(usd_aed_data['Date'], errors='coerce')

# Merge external datasets
merged_data = master_stock_data.copy()
merged_data = merged_data.merge(dfm_data[['Date', 'Price']], on='Date', how='left', suffixes=('', '_DFM'))
merged_data = merged_data.merge(brent_data[['Date', 'Price']], on='Date', how='left', suffixes=('', '_Brent'))
merged_data = merged_data.merge(usd_aed_data[['Date', 'Price']], on='Date', how='left', suffixes=('', '_USD_AED'))

# Rename columns for clarity
merged_data.rename(columns={
    'Price_DFM': 'DFM Index',
    'Price_Brent': 'Brent Oil Price',
    'Price_USD_AED': 'USD to AED Exchange Rate'
}, inplace=True)

# Handle missing values in external datasets using forward-fill
merged_data['DFM Index'] = merged_data['DFM Index'].ffill()
merged_data['Brent Oil Price'] = merged_data['Brent Oil Price'].ffill()
merged_data['USD to AED Exchange Rate'] = merged_data['USD to AED Exchange Rate'].ffill()

# Convert Brent Oil Price from USD to AED
merged_data['Brent Oil Price (AED)'] = merged_data['Brent Oil Price'] * merged_data['USD to AED Exchange Rate']
merged_data.drop(columns=['Brent Oil Price'], inplace=True)

# Save the processed data to a CSV file
merged_data.to_csv('enhanced_stock_data.csv', index=False)






In [3]:
merged_data

,Date,Price,Open,High,Low,Vol.,Change %,Bank Name,Price Lag 1,Change % Lag 1,Price MA 5,Price MA 10,Price per Vol,DFM Index,USD to AED Exchange Rate,Brent Oil Price (AED)
0,2014-12-17,6.000,5.550,6.250,5.050,10830000.0,9.29,ADCB,NaN,NaN,6.000,6.000,5.540166e-07,"3,033.00",3.6731,224.720258
1,2014-12-17,4.186,4.145,4.283,3.942,26450000.0,2.00,DIB,NaN,NaN,4.186,4.186,1.582609e-07,"3,033.00",3.6731,224.720258
2,2014-12-17,6.180,5.891,6.604,5.872,3130000.0,4.91,ENBD,NaN,NaN,6.180,6.180,1.974441e-06,"3,033.00",3.6731,224.720258
3,2014-12-17,9.860,9.090,10.000,9.090,662520.0,7.88,FAB,NaN,NaN,9.860,9.860,1.488257e-05,"3,033.00",3.6731,224.720258
4,2014-12-18,6.900,6.400,6.900,6.140,12710000.0,15.00,ADCB,6.00,9.29,6.450,6.450,5.428796e-07,"3,426.70",3.6731,217.704637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,2024-12-13,6.780,6.800,6.840,6.780,2000000.0,-0.29,DIB,6.80,0.59,6.788,6.809,3.390000e-06,"4,829.63",3.6724,273.557076
9877,2024-12-13,19.800,19.700,19.900,19.500,1680000.0,0.51,ENBD,19.70,1.03,19.600,19.620,1.178571e-05,"4,829.63",3.6724,273.557076
9878,2024-12-13,13.080,13.260,13.260,13.080,2730000.0,-1.36,FAB,13.26,0.15,13.224,13.122,4.791209e-06,"4,829.63",3.6724,273.557076
9879,2024-12-16,10.580,10.300,10.600,10.220,6390000.0,4.13,ADCB,10.16,0.40,10.110,10.059,1.655712e-06,"5,047.54",3.6729,271.464039


In [4]:
# Load GDP data
gdp_data = pd.read_csv('/Users/tanishq/Desktop/Projects/bank stock prices/gdp quartely.csv')


# Step 1: Filter for relevant data
gdp_filtered = gdp_data.loc[
    (gdp_data['MEASURE'] == 'TOT_GDP') & 
    (gdp_data['UNIT_MEASURE'] == 'MILAED') & 
    (gdp_data['QGDP_UNIT'] == 'VAL')
].copy()

# Step 2: Map QUARTER to starting months
quarter_mapping = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}
gdp_filtered['Start_Month'] = gdp_filtered['QUARTER'].map(quarter_mapping)

# Step 3: Combine TIME_PERIOD (year) with Start_Month to create a valid date
gdp_filtered['Date'] = pd.to_datetime(
    gdp_filtered['TIME_PERIOD'].astype(str) + '-' + gdp_filtered['Start_Month'] + '-01', 
    format='%Y-%m-%d'
)

# Step 4: Handle missing values in GDP data
# Fill missing values in OBS_VALUE with forward fill and backward fill
gdp_filtered['OBS_VALUE'] = gdp_filtered['OBS_VALUE'].ffill().bfill()

# Step 5: Expand GDP data to daily granularity
gdp_daily = gdp_filtered.set_index('Date').resample('D').ffill().reset_index()

# Step 6: Ensure master dataset's Date format matches
merged_data['Date'] = pd.to_datetime(merged_data['Date'], errors='coerce')

# Step 7: Merge GDP data with master dataset
merged_data = merged_data.merge(
    gdp_daily[['Date', 'OBS_VALUE']], 
    on='Date', 
    how='left'
)

# Rename the column for clarity
merged_data.rename(columns={'OBS_VALUE': 'GDP_Quarterly'}, inplace=True)

# Step 8: Identify missing GDP data
missing_dates = merged_data[merged_data['GDP_Quarterly'].isnull()]['Date'].unique()
print(f"Missing GDP data for these dates: {missing_dates}")

# Step 9: Resolve remaining missing values
# Fill remaining missing values with the mean GDP value
merged_data['GDP_Quarterly'] = merged_data['GDP_Quarterly'].fillna(merged_data['GDP_Quarterly'].mean())

# Step 10: Verify missing values are resolved
missing_gdp_after = merged_data['GDP_Quarterly'].isnull().sum()
print(f"Number of missing values in 'GDP_Quarterly' after filling: {missing_gdp_after}")





Missing GDP data for these dates: <DatetimeArray>
['2024-04-02 00:00:00', '2024-04-03 00:00:00', '2024-04-04 00:00:00',
 '2024-04-05 00:00:00', '2024-04-15 00:00:00', '2024-04-16 00:00:00',
 '2024-04-17 00:00:00', '2024-04-18 00:00:00', '2024-04-19 00:00:00',
 '2024-04-22 00:00:00',
 ...
 '2024-11-29 00:00:00', '2024-12-04 00:00:00', '2024-12-05 00:00:00',
 '2024-12-06 00:00:00', '2024-12-09 00:00:00', '2024-12-10 00:00:00',
 '2024-12-11 00:00:00', '2024-12-12 00:00:00', '2024-12-13 00:00:00',
 '2024-12-16 00:00:00']
Length: 176, dtype: datetime64[ns]
Number of missing values in 'GDP_Quarterly' after filling: 0


In [5]:
merged_data

,Date,Price,Open,High,Low,Vol.,Change %,Bank Name,Price Lag 1,Change % Lag 1,Price MA 5,Price MA 10,Price per Vol,DFM Index,USD to AED Exchange Rate,Brent Oil Price (AED),GDP_Quarterly
0,2014-12-17,6.000,5.550,6.250,5.050,10830000.0,9.29,ADCB,NaN,NaN,6.000,6.000,5.540166e-07,"3,033.00",3.6731,224.720258,303395.911300
1,2014-12-17,4.186,4.145,4.283,3.942,26450000.0,2.00,DIB,NaN,NaN,4.186,4.186,1.582609e-07,"3,033.00",3.6731,224.720258,303395.911300
2,2014-12-17,6.180,5.891,6.604,5.872,3130000.0,4.91,ENBD,NaN,NaN,6.180,6.180,1.974441e-06,"3,033.00",3.6731,224.720258,303395.911300
3,2014-12-17,9.860,9.090,10.000,9.090,662520.0,7.88,FAB,NaN,NaN,9.860,9.860,1.488257e-05,"3,033.00",3.6731,224.720258,303395.911300
4,2014-12-18,6.900,6.400,6.900,6.140,12710000.0,15.00,ADCB,6.00,9.29,6.450,6.450,5.428796e-07,"3,426.70",3.6731,217.704637,303395.911300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,2024-12-13,6.780,6.800,6.840,6.780,2000000.0,-0.29,DIB,6.80,0.59,6.788,6.809,3.390000e-06,"4,829.63",3.6724,273.557076,379226.258711
9877,2024-12-13,19.800,19.700,19.900,19.500,1680000.0,0.51,ENBD,19.70,1.03,19.600,19.620,1.178571e-05,"4,829.63",3.6724,273.557076,379226.258711
9878,2024-12-13,13.080,13.260,13.260,13.080,2730000.0,-1.36,FAB,13.26,0.15,13.224,13.122,4.791209e-06,"4,829.63",3.6724,273.557076,379226.258711
9879,2024-12-16,10.580,10.300,10.600,10.220,6390000.0,4.13,ADCB,10.16,0.40,10.110,10.059,1.655712e-06,"5,047.54",3.6729,271.464039,379226.258711


In [6]:
# Code to check missing values for all columns
missing_values_summary = merged_data.isnull().sum()

missing_values_summary


Date                        0
Price                       0
Open                        0
High                        0
Low                         0
Vol.                        0
Change %                    0
Bank Name                   0
Price Lag 1                 4
Change % Lag 1              4
Price MA 5                  0
Price MA 10                 0
Price per Vol               0
DFM Index                   0
USD to AED Exchange Rate    0
Brent Oil Price (AED)       0
GDP_Quarterly               0
dtype: int64

In [7]:
merged_data

,Date,Price,Open,High,Low,Vol.,Change %,Bank Name,Price Lag 1,Change % Lag 1,Price MA 5,Price MA 10,Price per Vol,DFM Index,USD to AED Exchange Rate,Brent Oil Price (AED),GDP_Quarterly
0,2014-12-17,6.000,5.550,6.250,5.050,10830000.0,9.29,ADCB,NaN,NaN,6.000,6.000,5.540166e-07,"3,033.00",3.6731,224.720258,303395.911300
1,2014-12-17,4.186,4.145,4.283,3.942,26450000.0,2.00,DIB,NaN,NaN,4.186,4.186,1.582609e-07,"3,033.00",3.6731,224.720258,303395.911300
2,2014-12-17,6.180,5.891,6.604,5.872,3130000.0,4.91,ENBD,NaN,NaN,6.180,6.180,1.974441e-06,"3,033.00",3.6731,224.720258,303395.911300
3,2014-12-17,9.860,9.090,10.000,9.090,662520.0,7.88,FAB,NaN,NaN,9.860,9.860,1.488257e-05,"3,033.00",3.6731,224.720258,303395.911300
4,2014-12-18,6.900,6.400,6.900,6.140,12710000.0,15.00,ADCB,6.00,9.29,6.450,6.450,5.428796e-07,"3,426.70",3.6731,217.704637,303395.911300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,2024-12-13,6.780,6.800,6.840,6.780,2000000.0,-0.29,DIB,6.80,0.59,6.788,6.809,3.390000e-06,"4,829.63",3.6724,273.557076,379226.258711
9877,2024-12-13,19.800,19.700,19.900,19.500,1680000.0,0.51,ENBD,19.70,1.03,19.600,19.620,1.178571e-05,"4,829.63",3.6724,273.557076,379226.258711
9878,2024-12-13,13.080,13.260,13.260,13.080,2730000.0,-1.36,FAB,13.26,0.15,13.224,13.122,4.791209e-06,"4,829.63",3.6724,273.557076,379226.258711
9879,2024-12-16,10.580,10.300,10.600,10.220,6390000.0,4.13,ADCB,10.16,0.40,10.110,10.059,1.655712e-06,"5,047.54",3.6729,271.464039,379226.258711


In [8]:
# Code to check missing values for all columns
missing_values_summary = merged_data.isnull().sum()

missing_values_summary


Date                        0
Price                       0
Open                        0
High                        0
Low                         0
Vol.                        0
Change %                    0
Bank Name                   0
Price Lag 1                 4
Change % Lag 1              4
Price MA 5                  0
Price MA 10                 0
Price per Vol               0
DFM Index                   0
USD to AED Exchange Rate    0
Brent Oil Price (AED)       0
GDP_Quarterly               0
dtype: int64

In [24]:
merged_data.to_csv('/Users/tanishq/Desktop/Projects/bank stock prices/master_stock_prices.csv', index=False)

In [10]:

interest_rate_path = '/Users/tanishq/Desktop/Projects/bank stock prices/interest rate 3csv.csv'
interest_rate = pd.read_csv(interest_rate_path)

# Ensure both 'Date' columns are datetime
merged_data['Date'] = pd.to_datetime(merged_data['Date'])
interest_rate['Date'] = pd.to_datetime(interest_rate['Date'])

# Perform the merge again
merged_data = pd.merge(merged_data, interest_rate, on='Date', how='left')
merged_data['Overnight'] = merged_data['Overnight'].fillna(merged_data['Overnight'].mean())

# Display the first few rows of the merged dataset
merged_data.head()



/var/folders/xz/q94zrrts0633czyfk72bc8tr0000gn/T/ipykernel_27787/2219396768.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  interest_rate['Date'] = pd.to_datetime(interest_rate['Date'])


,Date,Price,Open,High,Low,Vol.,Change %,Bank Name,Price Lag 1,Change % Lag 1,Price MA 5,Price MA 10,Price per Vol,DFM Index,USD to AED Exchange Rate,Brent Oil Price (AED),GDP_Quarterly,Overnight
0,2014-12-17,6.000,5.550,6.250,5.050,10830000.0,9.29,ADCB,NaN,NaN,6.000,6.000,5.540166e-07,"3,033.00",3.6731,224.720258,303395.9113,0.09286
1,2014-12-17,4.186,4.145,4.283,3.942,26450000.0,2.00,DIB,NaN,NaN,4.186,4.186,1.582609e-07,"3,033.00",3.6731,224.720258,303395.9113,0.09286
2,2014-12-17,6.180,5.891,6.604,5.872,3130000.0,4.91,ENBD,NaN,NaN,6.180,6.180,1.974441e-06,"3,033.00",3.6731,224.720258,303395.9113,0.09286
3,2014-12-17,9.860,9.090,10.000,9.090,662520.0,7.88,FAB,NaN,NaN,9.860,9.860,1.488257e-05,"3,033.00",3.6731,224.720258,303395.9113,0.09286
4,2014-12-18,6.900,6.400,6.900,6.140,12710000.0,15.00,ADCB,6.0,9.29,6.450,6.450,5.428796e-07,"3,426.70",3.6731,217.704637,303395.9113,0.09571


In [11]:
# Code to check missing values for all columns
missing_values_summary = merged_data.isnull().sum()

missing_values_summary

Date                        0
Price                       0
Open                        0
High                        0
Low                         0
Vol.                        0
Change %                    0
Bank Name                   0
Price Lag 1                 4
Change % Lag 1              4
Price MA 5                  0
Price MA 10                 0
Price per Vol               0
DFM Index                   0
USD to AED Exchange Rate    0
Brent Oil Price (AED)       0
GDP_Quarterly               0
Overnight                   0
dtype: int64

In [28]:
merged_data

,Date,Price,Open,High,Low,Vol.,Change %,Bank Name,Price Lag 1,Change % Lag 1,Price MA 5,Price MA 10,Price per Vol,DFM Index,USD to AED Exchange Rate,Brent Oil Price (AED),GDP_Quarterly,Overnight
0,2014-12-17,6.000,5.550,6.250,5.050,10830000.0,9.29,ADCB,NaN,NaN,6.000,6.000,5.540166e-07,"3,033.00",3.6731,224.720258,303395.911300,0.09286
1,2014-12-17,4.186,4.145,4.283,3.942,26450000.0,2.00,DIB,NaN,NaN,4.186,4.186,1.582609e-07,"3,033.00",3.6731,224.720258,303395.911300,0.09286
2,2014-12-17,6.180,5.891,6.604,5.872,3130000.0,4.91,ENBD,NaN,NaN,6.180,6.180,1.974441e-06,"3,033.00",3.6731,224.720258,303395.911300,0.09286
3,2014-12-17,9.860,9.090,10.000,9.090,662520.0,7.88,FAB,NaN,NaN,9.860,9.860,1.488257e-05,"3,033.00",3.6731,224.720258,303395.911300,0.09286
4,2014-12-18,6.900,6.400,6.900,6.140,12710000.0,15.00,ADCB,6.00,9.29,6.450,6.450,5.428796e-07,"3,426.70",3.6731,217.704637,303395.911300,0.09571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9928,2024-12-13,6.780,6.800,6.840,6.780,2000000.0,-0.29,DIB,6.80,0.59,6.788,6.809,3.390000e-06,"4,829.63",3.6724,273.557076,379226.258711,4.50567
9929,2024-12-13,19.800,19.700,19.900,19.500,1680000.0,0.51,ENBD,19.70,1.03,19.600,19.620,1.178571e-05,"4,829.63",3.6724,273.557076,379226.258711,4.50567
9930,2024-12-13,13.080,13.260,13.260,13.080,2730000.0,-1.36,FAB,13.26,0.15,13.224,13.122,4.791209e-06,"4,829.63",3.6724,273.557076,379226.258711,4.50567
9931,2024-12-16,10.580,10.300,10.600,10.220,6390000.0,4.13,ADCB,10.16,0.40,10.110,10.059,1.655712e-06,"5,047.54",3.6729,271.464039,379226.258711,4.56593
